In [ ]:
# Author: Adam Kim
# Date:   October 4, 2017
# Title:  CSI 4352, Assignment 4, Clustering by Agglomerative Approach

import os        as myOS    # get current directory
import csv       as myCSV   # handle initial file input

dIndex = 0  # Used for list indexing 
aIndex = 1  # Used for list indexing 
bIndex = 2  # Used for list indexing 

In [ ]:
# This function creates the gene -> dimensions dictionary.
# The gene index is the key, and the 12 datapoints for gene is the value

def init_DB(filename):

    handle          = open(filename,newline='')
    csvHandle       = myCSV.reader(handle,delimiter='\t',quotechar='"')

    myDB = {0:[]}
    myDB.pop(0,None)

    rowCount       = 0

    for row in csvHandle:

        for point in row:
            
            if point:

                if rowCount not in myDB:
                    myDB[rowCount] = [float(point)]
                else:
                    myDB[rowCount].append(float(point))

        rowCount += 1

    return myDB

In [ ]:
# Create cluster list
# For initial run, assign each gene to its own cluster
# There should be initially 500 clusters with 1 gene in each cluster

def init_clusters(myDB):

	clusters = []

	for gene in myDB:
		clusters.append([gene])

	return clusters

In [ ]:
# Build the distance matrix from gene A to gene B
# Here we use avg link distance between clusters
#   and manhattan distance between genes.
# For distance of gene to itself, insert infinity to prevent 0s on diagonal.

def init_distance_matrix(clusters,myDB):

    distMatrix = []

    i = 0

    while i < len(clusters):

        j = 0

        row = []

        while j < len(clusters):

            if i != j:

                avgLinkDist     = 0.0
                rootCluster     = clusters[i]
                neighborCluster = clusters[j]

                for gene1 in rootCluster:

                    for gene2 in neighborCluster:

                        avgLinkDist += manhattan(gene1,gene2,myDB)

                n           = float(len(rootCluster))
                m           = float(len(neighborCluster))
                avgLinkDist /= n
                avgLinkDist /= m
                
                row.append(avgLinkDist)
                
            else:

                row.append(float('inf'))

            j+=1

        distMatrix.append(row)

        i+=1
        
    return distMatrix

In [ ]:
# This function calculates the manhattan distance between two genes.

def manhattan(gene1,gene2,myDB):

    dimsA           = myDB[gene1]
    dimsB           = myDB[gene2]
    manhattanDist   = 0.0

    assert( len(dimsA) == len(dimsB) )

    itr = 0

    while itr < len(dimsA):

        manhattanDist += abs( dimsA[itr] - dimsB[itr] )

        itr += 1

    return manhattanDist

In [ ]:

################################################################################

# This is the AGNES algorithm.
# Start with clusters of size 1.
# Continuously merge the closest clusters until all member end up in 1 cluster.
# For this algorithm I will be reporting at cluster number 50,30, and 10

def AGNES(clusters,myDB):

    distMatrix = init_distance_matrix(clusters,myDB)

    # This is used to report results when cluster number is 50,30,10
    sizes = [50,30,10]
    
    while len(clusters) > 1:

        '''
        This code here outputs results when cluster number is 50,30,10
        '''
        if len(clusters) in sizes:
            sizes.remove(len(clusters))
            report(clusters,len(clusters))

        # Object used to track closest clusters
        minDist = [float('inf'),-1.0,-1.0]

        i=0
        while i < len(distMatrix):

            '''
            j=0
            
            This traverses the entire square.
            However, matrix is mirrored along diagonal.
            This is because distance is symmetric function.
            Distance from a to b is same distance from b to a
            Therefore duplicate information exists.
            
            To optimize, only read triangle.
            This reduces readtime from n^2 to 0.5n^2
            '''
            
            j = i # This traverses only the triangle
            
            while j < len(distMatrix):
                
                if distMatrix[i][j] < minDist[dIndex] and i != j:
                    
                    minDist[aIndex] = i
                    minDist[bIndex] = j
                    
                    minDist[dIndex] = distMatrix[i][j]
                    
                j+=1
                
            i+=1

        a = minDist[aIndex]
        b = minDist[bIndex]

        '''
        Here we merge cluster a with cluster b and delete cluster b
            within the clusters data structure.
        '''

        # Update a <- a+b, delete b in CLUSTERS
        clusters[a] = clusters[a] + clusters[b]
        del clusters[b]

        '''
        It is further necessary to update the distance matrix.

        First, delete the row containing cluster b's distance information
            because it is no longer needed
        '''
        
        # Delete b row in DISTANCE MATRIX
        del distMatrix[b]

        '''
        First, delete the row containing cluster b's distance information
            because it is no longer needed

            After these steps, the distance matrix size goes
                from n by n to n-1 by n-1
        '''
        
        # Delete b column in DISTANCE MATRIX
        for row in distMatrix:
            del row[b]
            
        rowA    = []
        k       = 0

        '''
        Here cluster b's distance information is fully removed
            from distance matrix.

        Now it is necessary to update distance information
            for cluster a (note a is merged with b)

        I will do this by created a row with recalculated distances
            from a to all other clusters (note b will not be calculated
            because it was removed in the previous step)
        '''

        rootCluster = clusters[a] # newly married cluster

        while k < len(clusters):

            if k != a:

                avgLinkDist     = 0.0
                neighborCluster = clusters[k]

                for gene1 in rootCluster:

                    for gene2 in neighborCluster:

                        avgLinkDist += manhattan(gene1,gene2,myDB)

                n           = float(len(rootCluster))
                m           = float(len(neighborCluster))
                avgLinkDist /= n
                avgLinkDist /= m

                rowA.append(avgLinkDist)

            else:
                
                rowA.append(float('inf'))

            k+=1

        # Here new row for a <- a+b built

        '''
        Here a row containing cluster a's new distances is created.
        Now it is necessary to update cluster a's row and column in the
            distance matrix.
        '''

        # Update cluster a's row in distance matrix
        distMatrix[a] = rowA

        # Update cluster a's column in distance matrix
        rowItr = 0
        while rowItr < len(distMatrix):
            distMatrix[rowItr][a] = rowA[rowItr]
            rowItr += 1

In [ ]:
# Driver for AGNES Algorithm.

def main():

    myDB        = init_DB(fetch_filename())

    clusters    = init_clusters(myDB)

    print('\nAGNES typically runs in 10 to 60 seconds depending on hardware.\n')

    print('Outputting to *_clusters_result-AdamKim.txt.\n')

    AGNES(clusters,myDB)


if __name__ == "__main__":
    main()
